# VegFrax Fractional Cover module tests

## Globals

In [12]:
# set globals paths
FOLDER_MODULES = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules'  
FOLDER_SHARED = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared'
TEST_MODULE = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\tests\code'
GRP_LYR_FILE = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\arc\lyr\group_template.lyrx'    

## Setup

### Imports

In [13]:
# imports
import os
import random
import numpy as np
import xarray as xr
import uuid
from IPython.utils import io

# import testing functions
sys.path.append(TEST_MODULE)
import test_funcs

# import full arcpy toolbox
arcpy.ImportToolbox(r"C:\Users\Lewis\Documents\GitHub\tenement-tools\arc\toolbox\tenement-tools-toolbox.pyt")

<module 'toolbox'>

### Reload libraries

In [14]:
# if scripts change, reload
from importlib import reload
reload(test_funcs)

<module 'test_funcs' from 'C:\\Users\\Lewis\\Documents\\GitHub\\tenement-tools\\tests\\code\\test_funcs.py'>

### Set data files and locations

In [7]:
# area a (tutorial area)

# set up area raw satellite options
in_ncs = [
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\tute_1_ls_10_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\tute_1_s2_18_20_raw_odc.nc"
]

# set current nc
in_nc = in_ncs[0]

# set high res geotiff
in_tifs = [
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\inputs\tutorial_area\veg_classified_1m.tif",
]

# set current tif
in_tif = in_tifs[0]

# set output temp nc, tif
out_nc = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\temp_nc.nc'
out_tif = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\temp_tif.tif'

In [7]:
# area b (roy hill)

# set up area raw satellite options
in_ncs = [
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\roy_1_ls_10_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\roy_3_ls_10_20_raw_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_likelihood\inputs\roy_2_s2_16_21_raw_odc.nc"
]

# set current nc
in_nc = in_ncs[0]

# set high res geotiff
in_tifs = [
    r"E:\Curtin\GDVII - General\Work Package 1\VegFrax\roy_hill\roy_veg_class.tif",
    r"E:\Curtin\GDVII - General\Work Package 1\VegFrax\roy_hill\roy_veg_broad_class.tif",
    r"E:\Curtin\GDVII - General\Work Package 1\VegFrax\VegFrax\veg_map.tif"
]

# set current tif
in_tif = in_tifs[0]

# set output temp nc, tif
out_nc = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\temp_nc.nc'
out_tif = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\temp_tif.tif'

## Testing

### Test: input low res netcdf

In [119]:
def test():
    
    classes = list(np.unique(xr.open_rasterio(in_tif)))
    if -999 in classes:
        classes.remove(-999)

    inputs = {
            'in_low_nc': '',
            'in_high_tif': '',
            'out_nc_file': out_nc,
            'in_aggregate_dates': True,
            'in_start_date': '1980/01/01',
            'in_end_date': '2050/12/31',
            'in_classes': ';'.join(['Class: {}'.format(c) for c in classes]),
            'in_aggregate_classes': False,
            'in_num_samples': 100,
            'in_max_nodata': 0.0,
            'in_smooth': False,
            'in_num_estimators': 100,
            'in_criterion': 'Mean Squared Error',
            'in_max_depth': None,
            'in_max_features': 'Auto',
            'in_boostrap': True,
            'in_fmask_flags': None,
            'in_max_cloud': 10,
            'in_add_result_to_map': True
        }
    
    test_msg = 'all default params'
    
    arcpy.env.addOutputsToMap = False

    try:
        print('\nTest: good nc, good tif, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            
            params.update({'in_low_nc': in_nc, 
                           'in_high_tif': in_tif})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e)
        
    raise
        
    try:
        print('\nTest: no x in nc, good tif, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.nc'
            out = out.format(str(uuid.uuid4().hex))
            with xr.open_dataset(in_nc) as ds: 
                ds.load()
            ds.drop('x').to_netcdf(out)

        params.update({'in_low_nc': out, 
                       'in_high_tif': in_tif})
        arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: no vars in nc, good tif, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.nc'
            out = out.format(str(uuid.uuid4().hex))
            with xr.open_dataset(in_nc) as ds: 
                ds.load()
            ds.drop_vars(ds.data_vars).to_netcdf(out)

            params.update({'in_low_nc': out, 
                           'in_high_tif': in_tif})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e)

    try:
        print('\nTest: 1 date in nc, good tif, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.nc'
            out = out.format(str(uuid.uuid4().hex))
            with xr.open_dataset(in_nc) as ds: 
                ds.load()
            ds.isel(time=slice(0, 1)).to_netcdf(out)

            params.update({'in_low_nc': out, 
                           'in_high_tif': in_tif})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e)

    try:
        print('\nTest: all nan nc, good tif, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.nc'
            out = out.format(str(uuid.uuid4().hex))
            with xr.open_dataset(in_nc) as ds: 
                ds.load()
            ds = xr.full_like(ds, fill_value=np.nan)
            ds.to_netcdf(out)

            params.update({'in_low_nc': out, 
                           'in_high_tif': in_tif})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e)

    try:
        print('\nTest: all 0 nc, good tif, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.nc'
            out = out.format(str(uuid.uuid4().hex))
            with xr.open_dataset(in_nc) as ds: 
                ds.load()
            ds = xr.full_like(ds, fill_value=0)
            ds.to_netcdf(out)

            params.update({'in_low_nc': out, 
                           'in_high_tif': in_tif})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e)

    try:
        print('\nTest: no attrs in nc, good tif, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.nc'
            out = out.format(str(uuid.uuid4().hex))
            with xr.open_dataset(in_nc) as ds: 
                ds.load()
            ds.attrs = {}
            ds.to_netcdf(out)

            params.update({'in_low_nc': out, 
                           'in_high_tif': in_tif})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e)

    try:
        print('\nTest: 300m res nc, good tif, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.nc'
            out = out.format(str(uuid.uuid4().hex))
            with xr.open_dataset(in_nc) as ds: 
                ds.load()
            ds.attrs = {}
            ds = ds.coarsen(x=10, y=10, boundary='trim').mean()
            ds.to_netcdf(out)

            params.update({'in_low_nc': out, 
                           'in_high_tif': in_tif})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: 0.8 res nc, good tif, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.nc'
            out = out.format(str(uuid.uuid4().hex))
            with xr.open_dataset(in_nc) as ds: 
                ds.load()
            attrs = ds.attrs
            ds = ds.isel(x=slice(0, 100), y=slice(0, 100), time=slice(0, 5))
            xs = np.linspace(ds['x'][0], ds['x'][-1], num=len(ds['x']) * 35)
            ys = np.linspace(ds['y'][0], ds['y'][-1], num=len(ds['y']) * 35)
            ds = ds.interp(x=xs, y=ys)
            ds.attrs = attrs
            ds.to_netcdf(out)

            params.update({'in_low_nc': out, 
                           'in_high_tif': in_tif})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: shifted 100km nc, good tif, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.nc'
            out = out.format(str(uuid.uuid4().hex))
            with xr.open_dataset(in_nc) as ds: 
                ds.load()
            ds['x'] = ds['x'] + 100000
            ds['y'] = ds['y'] + 100000
            ds.to_netcdf(out)

            params.update({'in_low_nc': out, 
                           'in_high_tif': in_tif})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: 10x10 pixel nc, good tif, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.nc'
            out = out.format(str(uuid.uuid4().hex))
            with xr.open_dataset(in_nc) as ds: 
                ds.load()
            ds = ds.isel(x=slice(0, 10), y=slice(0, 10))
            ds.to_netcdf(out)

            params.update({'in_low_nc': out, 
                           'in_high_tif': in_tif})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e)
        
    return

test()


Test: good nc, good tif, all default params.


RuntimeError: No active exception to reraise

### Test: input high res geotiff

In [16]:
def test():
    
    classes = list(np.unique(xr.open_rasterio(in_tif)))
    if -999 in classes:
        classes.remove(-999)

    inputs = {
            'in_low_nc': '',
            'in_high_tif': '',
            'out_nc_file': out_nc,
            'in_aggregate_dates': True,
            'in_start_date': '1980/01/01',
            'in_end_date': '2050/12/31',
            'in_classes': ';'.join(['Class: {}'.format(c) for c in classes]),
            'in_aggregate_classes': False,
            'in_num_samples': 100,
            'in_max_nodata': 0.0,
            'in_smooth': False,
            'in_num_estimators': 100,
            'in_criterion': 'Mean Squared Error',
            'in_max_depth': None,
            'in_max_features': 'Auto',
            'in_boostrap': True,
            'in_fmask_flags': None,
            'in_max_cloud': 10,
            'in_add_result_to_map': True
        }
    
    test_msg = 'all default params'
    
    arcpy.env.addOutputsToMap = False

    try:
        print('\nTest: good nc, good tif, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            
            params.update({'in_low_nc': in_nc, 
                           'in_high_tif': in_tif})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e)

    try:
        print('\nTest: good nc, tif shifted 5km, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))
            out = arcpy.management.Shift(in_tif, out, 5000, 5000)
            params.update({'in_low_nc': in_nc, 
                           'in_high_tif': out})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e) 

    try:
        print('\nTest: good nc, tif shifted 100km, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))
            out = arcpy.management.Shift(in_tif, out, 100000, 100000)
            params.update({'in_low_nc': in_nc, 
                           'in_high_tif': out})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e) 
        
    try:
        print('\nTest: good nc, tif in wgs84, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))
            out = arcpy.management.ProjectRaster(in_raster=in_tif, 
                                                 out_raster=out, 
                                                 out_coor_system=arcpy.SpatialReference(4326))
            params.update({'in_low_nc': in_nc, 
                           'in_high_tif': out})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e) 
        
    try:
        print('\nTest: good nc, tif all nodata, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))

            rm = arcpy.sa.RemapRange([[-9999, 9999, 'NoData']])
            lyr = arcpy.sa.Reclassify(in_tif, 'Value', rm)
            lyr.save(out)

            params.update({'in_low_nc': in_nc, 
                           'in_high_tif': out,
                           'in_classes': "'Class: 1'"})  # nodata defaults to 1
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e) 
        
    try:
        print('\nTest: good nc, tif all -999, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))

            rm = arcpy.sa.RemapRange([[-9999, 9999, -999]])
            lyr = arcpy.sa.Reclassify(in_tif, 'Value', rm)
            lyr.save(out)
            
            params.update({'in_low_nc': in_nc, 
                           'in_high_tif': out,
                           'in_classes': "'Class: -999'"})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e) 
        
    try:
        print('\nTest: good nc, tif all 0, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))

            rm = arcpy.sa.RemapRange([[-9999, 9999, 0]])
            lyr = arcpy.sa.Reclassify(in_tif, 'Value', rm)
            lyr.save(out)

            params.update({'in_low_nc': in_nc, 
                           'in_high_tif': out,
                           'in_classes': "'Class: 0'"})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e) 
        
    try:
        print('\nTest: good nc, tif 1 class is -999, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))

            rm = arcpy.sa.RemapRange([[1, 1, -999]])
            lyr = arcpy.sa.Reclassify(in_tif, 'Value', rm)
            lyr.save(out)

            params.update({'in_low_nc': in_nc, 
                           'in_high_tif': out,
                           'in_classes': "'Class: 2';'Class: 3';'Class: 4'"})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e) 
        
    try:
        print('\nTest: good nc, tif 1 class is 0, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))

            rm = arcpy.sa.RemapRange([[1, 1, 0]])
            lyr = arcpy.sa.Reclassify(in_tif, 'Value', rm)
            lyr.save(out)

            params.update({'in_low_nc': in_nc, 
                           'in_high_tif': out,
                           'in_classes': "'Class: 2';'Class: 3';'Class: 4'"})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e) 
           
    try:
        print('\nTest: good nc, tif 500mx500m, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))

            lyr = arcpy.sa.Raster(in_tif)
            ext = lyr.extent
            w, e = ext.XMin, ext.XMin + 500
            s, n = ext.YMin, ext.YMin + 500
            out = arcpy.management.Clip(in_raster=in_tif, 
                                        rectangle='{} {} {} {}'.format(w, s, e, n),
                                        out_raster=out)

            params.update({'in_low_nc': in_nc, 
                           'in_high_tif': out})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e) 
        
    try:
        print('\nTest: good nc, tif 250m res, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()            

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\vegfrax_fractional\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))
            
            out = arcpy.management.Resample(in_raster=in_tif, 
                                            out_raster=out, 
                                            cell_size="250 250")
            
            params.update({'in_low_nc': in_nc, 
                           'in_high_tif': out})
            arcpy.VegFrax_Fractional_Cover_toolbox(**params)
    except Exception as e:
        print(e) 
        
    return

test()


Test: good nc, tif 500mx500m, all default params.
Class 5 not within satellite NetCDF extent.
Failed to execute (VegFrax_Fractional_Cover).


Test: good nc, tif 250m res, all default params.
Could not build random samples, see messages for details.
Failed to execute (VegFrax_Fractional_Cover).



### Test: continuous and categorical geotiffs

In [6]:
def test():

    inputs = {
            'in_continuous_tifs': '',
            'in_categorical_tifs': '',
            'out_nc': out_nc,
            'in_occurrence_feat': in_shp,
            'in_num_pseudos': 1000,
            'in_equalise_pseudos': False,
            'in_test_ratio': 0.1,
            'in_resample': 'Lowest Resolution',
            'in_num_estimators': 100,
            'in_criterion': 'Gini',
            'in_max_depth': None,
            'in_max_features': 'Auto',
            'in_boostrap': True,
            'in_add_result_to_map': True
        }
    
    test_msg = 'all default params'
    
    arcpy.env.addOutputsToMap = False

    try:
        print('\nTest: good shp, many conts, many cates, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
    
    try:
        print('\nTest: good shp, one cont, one cate, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_continuous_tifs': cont_tifs[0], 
                           'in_categorical_tifs': cate_tifs[0]})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: good shp, one cont, no cate, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_continuous_tifs': cont_tifs[0], 
                           'in_categorical_tifs': ''})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: good shp, no cont, one cate, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_continuous_tifs': '', 
                           'in_categorical_tifs': cate_tifs[0]})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: good shp, one cont, one cate, both same tif, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_continuous_tifs': cont_tifs[0], 
                           'in_categorical_tifs': cont_tifs[0]})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: good shp, 2 random cont, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in random.sample(cont_tifs, 2)]), 
                           'in_categorical_tifs': ''})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)

    try:
        print('\nTest: good shp, 3 random cont, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in random.sample(cont_tifs, 3)]), 
                           'in_categorical_tifs': ''})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: good shp, 4 random cont, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in random.sample(cont_tifs, 4)]), 
                           'in_categorical_tifs': ''})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: good shp, no cont, 2 random cate, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_continuous_tifs': '', 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in random.sample(cont_tifs, 2)])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)    
        
    try:
        print('\nTest: good shp, one cont shifted 5k, {}.'.format(test_msg))
        with io.capture_output() as cap:
            _conts = cont_tifs.copy()
            params = inputs.copy()
            
            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.tif'
            out = out.format('_' + str(uuid.uuid4().hex))
            _conts[0] = arcpy.management.Shift(_conts[0], out, 5000, 5000)
            
            params.update({'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in _conts]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e) 
        
    try:
        print('\nTest: good shp, one cont shifted 100km, {}.'.format(test_msg))
        with io.capture_output() as cap:
            _conts = cont_tifs.copy()
            params = inputs.copy()
            
            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))
            _conts[0] = arcpy.management.Shift(_conts[0], out, 100000, 100000)

            params.update({'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in _conts]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: good shp, one cont in wgs84, {}.'.format(test_msg))
        with io.capture_output() as cap:
            _conts = cont_tifs.copy()
            params = inputs.copy()
            
            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))
            _conts[0] = arcpy.management.ProjectRaster(in_raster=_conts[0], 
                                                       out_raster=out, 
                                                       out_coor_system=arcpy.SpatialReference(4326))

            params.update({'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in _conts]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: good shp, one cont all NoData, {}.'.format(test_msg))
        with io.capture_output() as cap:
            _conts = cont_tifs.copy()
            params = inputs.copy()

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))

            rm = arcpy.sa.RemapRange([[-9999, 9999, 'NoData']])
            lyr = arcpy.sa.Reclassify(_conts[0], 'Value', rm)
            lyr.save(out)
            _conts[0] = out

            params.update({'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in _conts]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)

    try:
        print('\nTest: good shp, one cont some NoData, no cate, {}.'.format(test_msg))
        with io.capture_output() as cap:
            _conts = cont_tifs.copy()
            params = inputs.copy()

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))
            lyr = arcpy.ia.SetNull(_conts[0], _conts[0], "VALUE <= 180")
            lyr.save(out)
            _conts[0] = out

            params.update({'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in _conts])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)  
    
    try:
        print('\nTest: good shp, two vars, no valid values overlap, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.tif'
            out_a = out.format(str(uuid.uuid4().hex))
            out_b = out.format(str(uuid.uuid4().hex))

            rm = arcpy.sa.RemapRange([[1, 4, 'NoData']])
            lyr = arcpy.sa.Reclassify(cate_tifs[0], 'Value', rm)
            lyr.save(out_a)

            rm = arcpy.sa.RemapRange([[2, 5, 'NoData']])
            lyr = arcpy.sa.Reclassify(cate_tifs[0], 'Value', rm)
            lyr.save(out_b)           

            params.update({'in_continuous_tifs': out_a, 'in_categorical_tifs': out_b})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: good shp, all conts, cates, one cont 250m res, {}.'.format(test_msg))
        with io.capture_output() as cap:
            _conts = cont_tifs.copy()
            params = inputs.copy()
            
            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))
            _conts[0] = arcpy.management.Resample(in_raster=_conts[0], 
                                                  out_raster=out, 
                                                  cell_size="250 250")

            params.update({'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in _conts]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: shp in wgs84, all conts, cates, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.shp'
            out = out.format(str(uuid.uuid4().hex))
            out_srs = arcpy.SpatialReference(4326)
            arcpy.management.Project(in_dataset=in_shp, 
                                     out_dataset=out, 
                                     out_coor_system=out_srs)

            params.update({'in_occurrence_feat': out,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e) 
        
    try:
        print('\nTest: shp has no prj file, all conts, cates, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()
            
            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.shp'
            out = out.format(str(uuid.uuid4().hex))
            
            arcpy.management.CopyFeatures(in_features=in_shp, out_feature_class=out)
            os.remove(os.path.splitext(out)[0] + '.prj')

            params.update({'in_occurrence_feat': out,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e) 
        
    try:
        print('\nTest: shp has no points, all conts, cates, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()
            
            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.shp'
            out = out.format(str(uuid.uuid4().hex))
            
            arcpy.management.CopyFeatures(in_features=in_shp, out_feature_class=out)
            with arcpy.da.UpdateCursor(out, ['OID@']) as cursor:
                [cursor.deleteRow() for row in cursor]
            
            params.update({'in_occurrence_feat': out,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e) 

    try:
        print('\nTest: shp has 1% of total points, all conts, cates, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.shp'
            out = out.format(str(uuid.uuid4().hex))

            arcpy.ga.SubsetFeatures(in_features=in_shp,
                                    out_training_feature_class=out, 
                                    size_of_training_dataset=5,
                                    subset_size_units='PERCENTAGE_OF_INPUT')

            params.update({'in_occurrence_feat': out,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e) 
        
    try:
        print('\nTest: shp shifted 100km points, all conts, cates, {}.'.format(test_msg))
        with io.capture_output() as cap:
            params = inputs.copy()

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.shp'
            out = out.format(str(uuid.uuid4().hex))

            arcpy.management.CopyFeatures(in_features=in_shp, out_feature_class=out)
            with arcpy.da.UpdateCursor(out, ['SHAPE@XY']) as cursor:
                for row in cursor:
                    cursor.updateRow([[row[0][0] + (100000 or 0),
                                       row[0][1] + (100000 or 0)]])

            params.update({'in_occurrence_feat': out,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e) 

    return

test()


Test: good shp, many conts, many cates, all default params.

Test: good shp, one cont, one cate, all default params.

Test: good shp, one cont, no cate, all default params.
At least two variables are required.
Failed to execute (Nicher_SDM).


Test: good shp, no cont, one cate, all default params.
At least two variables are required.
Failed to execute (Nicher_SDM).


Test: good shp, one cont, one cate, both same tif, all default params.
Duplicate input GeoTiffs provided.
Failed to execute (Nicher_SDM).


Test: good shp, 2 random cont, all default params.

Test: good shp, 3 random cont, all default params.

Test: good shp, 4 random cont, all default params.

Test: good shp, no cont, 2 random cate, all default params.

Test: good shp, one cont shifted 5k, all default params.

Test: good shp, one cont shifted 100km, all default params.
Not all input layers intersect.
Failed to execute (Nicher_SDM).


Test: good shp, one cont in wgs84, all default params.
GeoTiff CRS is not EPSG:3577. EPS

### Test: number of absences

In [7]:
def test():

    inputs = {
            'in_continuous_tifs': '',
            'in_categorical_tifs': '',
            'out_nc': out_nc,
            'in_occurrence_feat': in_shp,
            'in_num_pseudos': 1000,
            'in_equalise_pseudos': False,
            'in_test_ratio': 0.1,
            'in_resample': 'Lowest Resolution',
            'in_num_estimators': 100,
            'in_criterion': 'Gini',
            'in_max_depth': None,
            'in_max_features': 'Auto',
            'in_boostrap': True,
            'in_add_result_to_map': True
        }
    
    test_msg = 'num pseudos: {}'
    
    arcpy.env.addOutputsToMap = False

    try:
        print('\nTest: good shp, many conts, many cates, {}.'.format(test_msg.format('default')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_pseudos': 1000,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
    
    try:
        print('\nTest: good shp, many conts, many cates, {}.'.format(test_msg.format('0')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_pseudos': 0,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)    
    
    try:
        print('\nTest: good shp, many conts, many cates, {}.'.format(test_msg.format('10')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_pseudos': 10,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)        
    
    try:
        print('\nTest: good shp, many conts, many cates, {}.'.format(test_msg.format('5000')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_pseudos': 5000,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)       

    try:
        print('\nTest: good shp, many conts, many cates, {}.'.format(test_msg.format('100000')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_pseudos': 100000,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)     

    try:
        print('\nTest: good shp, one cont some NoData, no cate, {}.'.format(test_msg.format('5000')))
        with io.capture_output() as cap:
            _conts = cont_tifs.copy()
            params = inputs.copy()

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))
            lyr = arcpy.ia.SetNull(_conts[0], _conts[0], "VALUE <= 180")
            lyr.save(out)
            _conts[0] = out

            params.update({'in_num_pseudos': 5000,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in _conts])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)  
    
    try:
        print('\nTest: good shp, all conts, cates, one cont 250m res, {}.'.format(test_msg.format('1000')))
        with io.capture_output() as cap:
            _conts = cont_tifs.copy()
            params = inputs.copy()
            
            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))
            _conts[0] = arcpy.management.Resample(in_raster=_conts[0], 
                                                  out_raster=out, 
                                                  cell_size="250 250")

            
            params.update({'in_num_pseudos': 1000,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in _conts]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: shp has 1% of total points, all conts, cates, {}.'.format(test_msg.format('2500')))
        with io.capture_output() as cap:
            params = inputs.copy()

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.shp'
            out = out.format(str(uuid.uuid4().hex))

            arcpy.ga.SubsetFeatures(in_features=in_shp,
                                    out_training_feature_class=out, 
                                    size_of_training_dataset=5,
                                    subset_size_units='PERCENTAGE_OF_INPUT')

            params.update({'in_num_pseudos': 2500,
                           'in_occurrence_feat': out,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e) 

    return

test()


Test: good shp, many conts, many cates, num pseudos: default.

Test: good shp, many conts, many cates, num pseudos: 0.
Failed to execute. Parameters are not valid.
WARNING 000725: Output Nicher NetCDF file: Dataset E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\temp_nc.nc already exists.
ERROR 000854: The value is out of the range from 1 to 1e+07.
Failed to execute (Nicher_SDM).


Test: good shp, many conts, many cates, num pseudos: 10.

Test: good shp, many conts, many cates, num pseudos: 5000.

Test: good shp, many conts, many cates, num pseudos: 100000.

Test: good shp, one cont some NoData, no cate, num pseudos: 5000.
Input GeoTiff is completely null.
Failed to execute (Nicher_SDM).


Test: good shp, all conts, cates, one cont 250m res, num pseudos: 1000.

Test: shp has 1% of total points, all conts, cates, num pseudos: 2500.


### Test: equalise absences

In [39]:
def test():

    inputs = {
            'in_continuous_tifs': '',
            'in_categorical_tifs': '',
            'out_nc': out_nc,
            'in_occurrence_feat': in_shp,
            'in_num_pseudos': 1000,
            'in_equalise_pseudos': True,
            'in_test_ratio': 0.1,
            'in_resample': 'Lowest Resolution',
            'in_num_estimators': 100,
            'in_criterion': 'Gini',
            'in_max_depth': None,
            'in_max_features': 'Auto',
            'in_boostrap': True,
            'in_add_result_to_map': True
        }
    
    test_msg = 'num pseudos: {}'
    
    arcpy.env.addOutputsToMap = False

    try:
        print('\nTest: good shp, many conts, many cates, {}.'.format(test_msg.format('default with equalise on')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_pseudos': 1000,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
    
    try:
        print('\nTest: good shp, many conts, many cates, {}.'.format(test_msg.format('0 with equalise on')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_pseudos': 0,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)    
    
    try:
        print('\nTest: good shp, many conts, many cates, {}.'.format(test_msg.format('10 with equalise on')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_pseudos': 10,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)        
    
    try:
        print('\nTest: good shp, many conts, many cates, {}.'.format(test_msg.format('5000 with equalise on')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_pseudos': 5000,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)       

    try:
        print('\nTest: good shp, one cont some NoData, no cate, {}.'.format(test_msg.format('5000 with equalise on')))
        with io.capture_output() as cap:
            _conts = cont_tifs.copy()
            params = inputs.copy()

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))
            lyr = arcpy.ia.SetNull(_conts[0], _conts[0], "VALUE <= 180")
            lyr.save(out)
            _conts[0] = out

            params.update({'in_num_pseudos': 5000,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in _conts])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)  
    
    try:
        print('\nTest: good shp, all conts, cates, one cont 250m res, {}.'.format(test_msg.format('1000 with equalise on')))
        with io.capture_output() as cap:
            _conts = cont_tifs.copy()
            params = inputs.copy()
            
            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))
            _conts[0] = arcpy.management.Resample(in_raster=_conts[0], 
                                                  out_raster=out, 
                                                  cell_size="250 250")

            
            params.update({'in_num_pseudos': 1000,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in _conts]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: shp has 1% of total points, all conts, cates, {}.'.format(test_msg.format('2500 with equalise on')))
        with io.capture_output() as cap:
            params = inputs.copy()

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.shp'
            out = out.format(str(uuid.uuid4().hex))

            arcpy.ga.SubsetFeatures(in_features=in_shp,
                                    out_training_feature_class=out, 
                                    size_of_training_dataset=5,
                                    subset_size_units='PERCENTAGE_OF_INPUT')

            params.update({'in_num_pseudos': 2500,
                           'in_occurrence_feat': out,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e) 

    return

test()


Test: good shp, many conts, many cates, num pseudos: default with equalise on.

Test: good shp, many conts, many cates, num pseudos: 0 with equalise on.
Failed to execute. Parameters are not valid.
WARNING 000725: Output Nicher NetCDF file: Dataset E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\temp_nc.nc already exists.
ERROR 000854: The value is out of the range from 1 to 1e+07.
Failed to execute (Nicher_SDM).


Test: good shp, many conts, many cates, num pseudos: 10 with equalise on.

Test: good shp, many conts, many cates, num pseudos: 5000 with equalise on.

Test: good shp, one cont some NoData, no cate, num pseudos: 5000 with equalise on.
Input GeoTiff is completely null.
Failed to execute (Nicher_SDM).


Test: good shp, all conts, cates, one cont 250m res, num pseudos: 1000 with equalise on.

Test: shp has 1% of total points, all conts, cates, num pseudos: 2500 with equalise on.


### Test: test ratio

In [9]:
def test():

    inputs = {
            'in_continuous_tifs': '',
            'in_categorical_tifs': '',
            'out_nc': out_nc,
            'in_occurrence_feat': in_shp,
            'in_num_pseudos': 1000,
            'in_equalise_pseudos': False,
            'in_test_ratio': 0.1,
            'in_resample': 'Lowest Resolution',
            'in_num_estimators': 100,
            'in_criterion': 'Gini',
            'in_max_depth': None,
            'in_max_features': 'Auto',
            'in_boostrap': True,
            'in_add_result_to_map': True
        }
    
    test_msg = 'ratio: {}'
    
    arcpy.env.addOutputsToMap = False

    try:
        print('\nTest: good shp, many conts, many cates, {}.'.format(test_msg.format('0.1')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_pseudos': 1000,
                           'in_test_ratio': 0.1,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: good shp, many conts, many cates, 1000 abse {}.'.format(test_msg.format('0')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_pseudos': 1000,
                           'in_test_ratio': 0,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: good shp, many conts, many cates, 1000 abse {}.'.format(test_msg.format('0.9')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_pseudos': 1000,
                           'in_test_ratio': 0.9,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)        
        
    try:
        print('\nTest: good shp, many conts, many cates, 1000 abse {}.'.format(test_msg.format('0.1')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_pseudos': 1000,
                           'in_test_ratio': 0.1,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: good shp, many conts, many cates, 100 abse {}.'.format(test_msg.format('0.1')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_pseudos': 100,
                           'in_test_ratio': 0.1,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)            
        
    try:
        print('\nTest: good shp, many conts, many cates, 100 abse {}.'.format(test_msg.format('0.9')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_pseudos': 100,
                           'in_test_ratio': 0.9,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)    

    try:
        print('\nTest: good shp, many conts, no cates, 5000 abse {}.'.format(test_msg.format('0.5')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_pseudos': 5000,
                           'in_test_ratio': 0.5,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ''})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e) 
        
    try:
        print('\nTest: good shp, many conts, many cates, 100 abse {}, equalise on.'.format(test_msg.format('0.9')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_pseudos': 100,
                           'in_test_ratio': 0.9,
                           'in_equalise_pseudos': True,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)        

    try:
        print('\nTest: good shp, many conts, many cates, 5000 abse {}, equalise on.'.format(test_msg.format('0.2')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_pseudos': 5000,
                           'in_test_ratio': 0.2,
                           'in_equalise_pseudos': True,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)    
        
    try:
        print('\nTest: good shp, many conts, no cates, 5000 abse {}, equalise on.'.format(test_msg.format('0.2')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_pseudos': 5000,
                           'in_test_ratio': 0.2,
                           'in_equalise_pseudos': True,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ''})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)      

    try:
        print('\nTest: good shp, all conts, no cates, one cont 250m res, 500 abse, {}.'.format(test_msg.format('0.4')))
        with io.capture_output() as cap:
            _conts = cont_tifs.copy()
            params = inputs.copy()
            
            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))
            _conts[0] = arcpy.management.Resample(in_raster=_conts[0], 
                                                  out_raster=out, 
                                                  cell_size="250 250")

            
            params.update({'in_num_pseudos': 1000,
                           'in_test_ratio': 0.4,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in _conts]), 
                           'in_categorical_tifs': ''})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)

    try:
        print('\nTest: shp has 1% of total points, all conts, cates, 2500 abse, {}.'.format(test_msg.format('0.5')))
        with io.capture_output() as cap:
            params = inputs.copy()

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.shp'
            out = out.format(str(uuid.uuid4().hex))

            arcpy.ga.SubsetFeatures(in_features=in_shp,
                                    out_training_feature_class=out, 
                                    size_of_training_dataset=5,
                                    subset_size_units='PERCENTAGE_OF_INPUT')

            params.update({'in_num_pseudos': 2500,
                           'in_test_ratio': 0.5,
                           'in_occurrence_feat': out,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e) 

    return

test()


Test: good shp, many conts, many cates, ratio: 0.1.

Test: good shp, many conts, many cates, 1000 abse ratio: 0.
Testing ratio must be between 0 and 1.
Failed to execute (Nicher_SDM).


Test: good shp, many conts, many cates, 1000 abse ratio: 0.9.

Test: good shp, many conts, many cates, 1000 abse ratio: 0.1.

Test: good shp, many conts, many cates, 100 abse ratio: 0.1.

Test: good shp, many conts, many cates, 100 abse ratio: 0.9.

Test: good shp, many conts, no cates, 5000 abse ratio: 0.5.

Test: good shp, many conts, many cates, 100 abse ratio: 0.9, equalise on.

Test: good shp, many conts, many cates, 5000 abse ratio: 0.2, equalise on.

Test: good shp, many conts, no cates, 5000 abse ratio: 0.2, equalise on.

Test: good shp, all conts, no cates, one cont 250m res, 500 abse, ratio: 0.4.

Test: shp has 1% of total points, all conts, cates, 2500 abse, ratio: 0.5.


### Test: resample

In [11]:
def test():

    inputs = {
            'in_continuous_tifs': '',
            'in_categorical_tifs': '',
            'out_nc': out_nc,
            'in_occurrence_feat': in_shp,
            'in_num_pseudos': 1000,
            'in_equalise_pseudos': False,
            'in_test_ratio': 0.1,
            'in_resample': 'Lowest Resolution',
            'in_num_estimators': 100,
            'in_criterion': 'Gini',
            'in_max_depth': None,
            'in_max_features': 'Auto',
            'in_boostrap': True,
            'in_add_result_to_map': True
        }
    
    test_msg = 'resample {}'
    
    arcpy.env.addOutputsToMap = False

    try:
        print('\nTest: good shp, many conts, many cates, {}.'.format(test_msg.format('default')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_resample': 'Lowest Resolution',
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: good shp, many conts, many cates, {}.'.format(test_msg.format('empty ')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_resample': '',
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
    
    try:
        print('\nTest: good shp, many conts, many cates, {}.'.format(test_msg.format('highest')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_resample': 'Highest Resolution',
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: good shp, three cont, no cate, {}.'.format(test_msg.format('lowest')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_resample': 'Lowest Resolution',
                           'in_continuous_tifs': cont_tifs[0:3], 
                           'in_categorical_tifs': ''})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: good shp, three cont, no cate, {}.'.format(test_msg.format('highest')))
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_resample': 'Highest Resolution',
                           'in_continuous_tifs': cont_tifs[0:3], 
                           'in_categorical_tifs': ''})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)

    try:
        print('\nTest: good shp, all conts, cates, one cont 250m res, {}.'.format(test_msg.format('lowest')))
        with io.capture_output() as cap:
            _conts = cont_tifs.copy()
            params = inputs.copy()
            
            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))
            _conts[0] = arcpy.management.Resample(in_raster=_conts[0], 
                                                  out_raster=out, 
                                                  cell_size="250 250")

            params.update({'in_resample': 'Lowest Resolution',
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in _conts]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: good shp, all conts, cates, one cont 250m res, {}.'.format(test_msg.format('highest')))
        with io.capture_output() as cap:
            _conts = cont_tifs.copy()
            params = inputs.copy()
            
            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))
            _conts[0] = arcpy.management.Resample(in_raster=_conts[0], 
                                                  out_raster=out, 
                                                  cell_size="250 250")

            params.update({'in_resample': 'Highest Resolution',
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in _conts]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)       
        
    try:
        print('\nTest: good shp, all conts, cates, one cont 1m res, {}.'.format(test_msg.format('lowest')))
        with io.capture_output() as cap:
            _conts = cont_tifs.copy()
            params = inputs.copy()
            
            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))
            _conts[0] = arcpy.management.Resample(in_raster=_conts[0], 
                                                  out_raster=out, 
                                                  cell_size="1 1")

            params.update({'in_resample': 'Lowest Resolution',
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in _conts]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
        
    try:
        print('\nTest: good shp, all conts, cates, one cont 1m res, {}.'.format(test_msg.format('highest')))
        with io.capture_output() as cap:
            _conts = cont_tifs.copy()
            params = inputs.copy()

            out = r'E:\Curtin\GDVII - General\Work Package 2\test_data\nicher_sdm\outputs\{}.tif'
            out = out.format(str(uuid.uuid4().hex))
            _conts[0] = arcpy.management.Resample(in_raster=_conts[0], 
                                                  out_raster=out, 
                                                  cell_size="1 1")

            params.update({'in_resample': 'Highest Resolution',
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in _conts]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
        except Exception as e:
            print(e)    

    return

test()

IndentationError: unexpected indent (<string>, line 138)

### Test: model parameters

In [ ]:
def test():

    inputs = {
            'in_continuous_tifs': '',
            'in_categorical_tifs': '',
            'out_nc': out_nc,
            'in_occurrence_feat': in_shp,
            'in_num_pseudos': 1000,
            'in_equalise_pseudos': False,
            'in_test_ratio': 0.1,
            'in_resample': 'Lowest Resolution',
            'in_num_estimators': 100,
            'in_criterion': 'Gini',
            'in_max_depth': None,
            'in_max_features': 'Auto',
            'in_boostrap': True,
            'in_add_result_to_map': True
        }
        
    arcpy.env.addOutputsToMap = False
    
    # ests 100, crit gini, mdep none, mfeatm auto boot true

    try:
        print('\nTest: good shp, many conts, many cates, all default ests, crit, mdep, mfeatm boot')
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)
    
    try:
        print('\nTest: good shp, many conts, many cates, ests 1000')
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_estimators': 1000,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)    

    try:
        print('\nTest: good shp, many conts, many cates, ests 10')
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_num_estimators': 10,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)    
    
    
    try:
        print('\nTest: good shp, many conts, many cates, crit gini')
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_criterion': 'Gini',
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)    

    try:
        print('\nTest: good shp, many conts, many cates, crit entropy')
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_criterion': 'Entropy',
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)      
    
    try:
        print('\nTest: good shp, many conts, many cates, mdepth none')
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_max_depth': None,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)      

    try:
        print('\nTest: good shp, many conts, many cates, mdepth 1000')
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_max_depth': 1000,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)  

    try:
        print('\nTest: good shp, many conts, many cates, mdepth -5')
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_max_depth': -5,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e) 
    
    try:
        print('\nTest: good shp, many conts, many cates, mfeats auto')
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_max_features': 'Auto',
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)      
    
    try:
        print('\nTest: good shp, many conts, many cates, mfeats log2')
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_max_features': 'Log2',
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)       

    try:
        print('\nTest: good shp, many conts, many cates, boostrap true')
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_boostrap': True,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)  

    try:
        print('\nTest: good shp, many conts, many cates, boostrap false')
        with io.capture_output() as cap:
            params = inputs.copy()
            params.update({'in_boostrap': False,
                           'in_continuous_tifs': ';'.join(["'{}'".format(t) for t in cont_tifs]), 
                           'in_categorical_tifs': ';'.join(["'{}'".format(t) for t in cate_tifs])})
            arcpy.Nicher_SDM_toolbox(**params)
    except Exception as e:
        print(e)  

    return

test()